In [1]:
import pandas
import numpy
import matplotlib


In [2]:
dataframe_ratings = pandas.read_csv('ml-32m/ratings.csv')
dataframe_tags = pandas.read_csv('ml-32m/tags.csv')
dataframe_movies = pandas.read_csv('ml-32m/movies.csv')

In [3]:
dataframe_ratings.head()

,userId,movieId,rating,timestamp
0,1,17,4.0,944249077
1,1,25,1.0,944250228
2,1,29,2.0,943230976
3,1,30,5.0,944249077
4,1,32,5.0,943228858


In [4]:
dataframe_tags.head()

,userId,movieId,tag,timestamp
0,22,26479,Kevin Kline,1583038886
1,22,79592,misogyny,1581476297
2,22,247150,acrophobia,1622483469
3,34,2174,music,1249808064
4,34,2174,weird,1249808102


In [5]:
dataframe_movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [6]:
dataframe_ratings_con_genres = dataframe_ratings.merge(
    dataframe_movies[['movieId', 'genres']], 
    on='movieId', 
    how='left'
)


In [7]:
# Verificar si hay filas con valores NA
filas_con_na = dataframe_ratings_con_genres.isnull().any(axis=1).sum()
print(f"Total de filas con valores NA: {filas_con_na}")

# Ver valores NA por columna
print("\nValores NA por columna:")
print(dataframe_ratings_con_genres.isnull().sum())

# Mostrar algunas filas con NA si existen
if filas_con_na > 0:
    print("\nPrimeras filas con valores NA:")
    print(dataframe_ratings_con_genres[dataframe_ratings_con_genres.isnull().any(axis=1)].head())


Total de filas con valores NA: 0

Valores NA por columna:
userId       0
movieId      0
rating       0
timestamp    0
genres       0
dtype: int64


In [8]:
# Extraer todos los géneros únicos
todos_generos = set()
for generos_str in dataframe_ratings_con_genres['genres'].dropna():
    if generos_str != '(no genres listed)':
        todos_generos.update(generos_str.split('|'))

# Crear diccionario de clasificación (género -> entero)
generos_sorted = sorted(list(todos_generos))
diccionario_generos = {genero: idx for idx, genero in enumerate(generos_sorted)}

# Convertir géneros de strings a enteros
def convertir_generos_a_enteros(generos_str):
    if pandas.isna(generos_str) or generos_str == '(no genres listed)':
        return []
    return [diccionario_generos[g] for g in generos_str.split('|')]

dataframe_ratings_con_genres['genres_enteros'] = dataframe_ratings_con_genres['genres'].apply(convertir_generos_a_enteros)


In [9]:
dataframe_ratings_con_genres.head()

,userId,movieId,rating,timestamp,genres,genres_enteros
0,1,17,4.0,944249077,Drama|Romance,"[7, 14]"
1,1,25,1.0,944250228,Drama|Romance,"[7, 14]"
2,1,29,2.0,943230976,Adventure|Drama|Fantasy|Mystery|Sci-Fi,"[1, 7, 8, 13, 15]"
3,1,30,5.0,944249077,Crime|Drama,"[5, 7]"
4,1,32,5.0,943228858,Mystery|Sci-Fi|Thriller,"[13, 15, 16]"


In [10]:
import numpy as np
from scipy.sparse import csr_matrix

In [11]:
# Normalización de ratings (centrada por usuario)
# Calculamos el promedio de rating de cada usuario
promedios_usuario = dataframe_ratings_con_genres.groupby('userId')['rating'].transform('mean')

# Creamos una nueva columna con ratings normalizados
dataframe_ratings_con_genres['rating_normalizado'] = dataframe_ratings_con_genres['rating'] - promedios_usuario

# Mostrar ejemplo de normalización
print("\nEjemplo de ratings normalizados:")
print(dataframe_ratings_con_genres[['userId', 'movieId', 'rating', 'rating_normalizado']].head())


Ejemplo de ratings normalizados:
   userId  movieId  rating  rating_normalizado
0       1       17     4.0            0.468085
1       1       25     1.0           -2.531915
2       1       29     2.0           -1.531915
3       1       30     5.0            1.468085
4       1       32     5.0            1.468085


In [12]:
# Generación de matriz usuario–película en formato disperso (sparse)
# Convertimos los IDs a índices numéricos para construir la matriz
user_codes = dataframe_ratings_con_genres['userId'].astype('category').cat.codes
movie_codes = dataframe_ratings_con_genres['movieId'].astype('category').cat.codes

# Creamos la matriz dispersa usando los ratings normalizados
matriz_dispersa = csr_matrix((
    dataframe_ratings_con_genres['rating_normalizado'],
    (user_codes, movie_codes)
))

# Mostramos la información de la matriz dispersa
print("\nMatriz dispersa creada correctamente.")
print(f"Forma: {matriz_dispersa.shape[0]} usuarios x {matriz_dispersa.shape[1]} películas")
print(f"Porcentaje de celdas ocupadas: {matriz_dispersa.nnz / (matriz_dispersa.shape[0]*matriz_dispersa.shape[1])*100:.6f}%")


Matriz dispersa creada correctamente.
Forma: 200948 usuarios x 84432 películas
Porcentaje de celdas ocupadas: 0.188609%


In [13]:
# Generar versión reducida (Para mostrar a modo de ejemplo)
# Seleccionamos una muestra pequeña de usuarios y películas
usuarios_muestra = dataframe_ratings_con_genres['userId'].drop_duplicates().head(5)
peliculas_muestra = dataframe_ratings_con_genres['movieId'].drop_duplicates().head(10)

# Creamos una matriz pequeña con pivot_table (solo para mostrar)
matriz_muestra = dataframe_ratings_con_genres[
    dataframe_ratings_con_genres['userId'].isin(usuarios_muestra) &
    dataframe_ratings_con_genres['movieId'].isin(peliculas_muestra)
].pivot_table(
    index='userId',
    columns='movieId',
    values='rating_normalizado'
).fillna('—')

print("\nMatriz de ejemplo (para incluir en el informe):")
display(matriz_muestra)


Matriz de ejemplo (para incluir en el informe):


movieId,17,25,29,30,32,34,36,80,110,111
userId,,,,,,,,,,
1,0.468085,-2.531915,-1.531915,1.468085,1.468085,-1.531915,-2.531915,1.468085,-0.531915,1.468085
2,—,—,—,—,—,0.730769,—,—,—,—
3,1.411565,—,—,—,—,—,—,—,1.411565,—
5,—,—,—,—,—,—,—,—,0.727273,—


### GUARDAR MATRIZ DISPERSA PARA USO FUTURO

In [ ]:
from scipy.sparse import save_npz, load_npz
import os
import pandas as pd

# Guardar la matriz dispersa en formato .npz
save_npz("matriz_usuario_pelicula.npz", matriz_dispersa)
print("\nMatriz dispersa guardada correctamente en 'matriz_usuario_pelicula.npz'.")

# Guardar también el mapeo entre códigos e IDs reales
usuarios = dataframe_ratings_con_genres['userId'].astype('category')
peliculas = dataframe_ratings_con_genres['movieId'].astype('category')

usuarios_index = pd.DataFrame({
    'user_index': range(len(usuarios.cat.categories)),
    'userId': usuarios.cat.categories
})
peliculas_index = pd.DataFrame({
    'movie_index': range(len(peliculas.cat.categories)),
    'movieId': peliculas.cat.categories
})

usuarios_index.to_csv("mapa_usuarios.csv", index=False)
peliculas_index.to_csv("mapa_peliculas.csv", index=False)
print("Mapas de índices guardados (usuarios y películas).")


Matriz dispersa guardada correctamente en 'matriz_usuario_pelicula.npz'.
Mapas de índices guardados (usuarios y películas).
